In [1]:
import duckdb

In [2]:
con = duckdb.connect(database=':memory:', read_only=False)

In [3]:
query = "SELECT * from 'customers.csv'"

In [4]:
result = con.execute(query).fetchdf()

In [5]:
result.head(2)

,customer_id,first_name,last_name,email,created_at,created_date,status,country
0,1001,John,Doe,john.doe@example.com,2024-01-15 08:23:45,2024-01-15,active,US
1,1002,Emily,Smith,emily.smith@example.co.uk,2024-02-10 14:12:30,2024-02-10,active,UK


In [6]:
result.shape

(10, 8)

In [69]:
filename = "Titanic-Dataset.csv"

In [70]:
query = f"select * from '{filename}'"

In [71]:
titatic_result = con.execute(query).fetchdf()

In [65]:
titatic_result.head()

,customer_id,customer_name,segment,order_count,total_revenue,avg_order_value
0,C002,Jane Smith,Corporate,2,350.0,175.00
1,C003,Acme Corp,SMB,1,80.0,80.00
2,C010,Small Biz,SMB,1,45.0,45.00
3,C001,John Doe,Consumer,2,195.5,97.75
4,C005,Global LLC,Enterprise,2,800.0,400.00


In [48]:
titatic_result.shape

(8, 6)

In [72]:
titatic_result.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [73]:
query = "select * from read_json_auto('products.json')"

In [74]:
product_result = con.execute(query).fetchdf()

In [75]:
product_result.head()

,id,name,price,category,in_stock
0,1,Laptop,35000,Electronics,True
1,2,Phone,18000,Electronics,False
2,3,Desk Chair,4500,Furniture,True


In [76]:
result.columns

Index(['customer_id', 'first_name', 'last_name', 'email', 'created_at',
       'created_date', 'status', 'country'],
      dtype='object')

In [77]:
query = """
SELECT
  customer_id,
  first_name,
  last_name,
  email,
  country,
  created_date
FROM 'customers.csv'
WHERE status = 'active'
  AND created_date >= '2024-01-01'
  AND country IN ('TH');
"""

thai_customers = con.execute(query).fetchdf()

In [78]:
customers_result2 = con.execute(query).fetchdf()

In [79]:
customers_result2.head()

,customer_id,first_name,last_name,email,country,created_date
0,1004,Sophia,Wilson,sophia.wilson@example.co.th,TH,2024-04-01
1,1008,Amanda,Lee,amanda.lee@example.co.th,TH,2024-09-02
2,1010,Nicha,Chaiyaphum,nicha.chaiyaphum@example.co.th,TH,2025-02-14


In [80]:
titatic_result.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [16]:
query = """
SELECT
    PassengerId,
    Name,
    Age,
    Fare,
    CASE
        WHEN Survived = 1 THEN 'Survived'
        ELSE 'Not Survived'
    END as SurvivalStatus
FROM 'Titanic-Dataset.csv'
"""

transformed_data = con.execute(query).fetchdf()

In [17]:
transformed_data.head()

,PassengerId,Name,Age,Fare,SurvivalStatus
0,1,"Braund, Mr. Owen Harris",22.0,7.2500,Not Survived
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,71.2833,Survived
2,3,"Heikkinen, Miss. Laina",26.0,7.9250,Survived
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,53.1000,Survived
4,5,"Allen, Mr. William Henry",35.0,8.0500,Not Survived


In [18]:
query = "select * from 'orders.csv'"

In [19]:
query = "select * from 'new_customers.csv'"

In [39]:
query = """
SELECT
    c.customer_id,
    c.customer_name,
    c.segment,
    COUNT(DISTINCT o.order_id) as order_count,
    SUM(o.amount) as total_revenue,
    AVG(o.amount) as avg_order_value
FROM new_customers.csv c
LEFT JOIN orders.csv o
    ON c.customer_id = o.customer_id
    AND o.status != 'cancelled'
WHERE c.is_active = true
GROUP BY 1, 2, 3
"""

In [82]:
customer_metric = con.execute(query).fetchdf()

In [81]:
customer_metric.head()

,customer_id,customer_name,segment,order_count,total_revenue,avg_order_value
0,C002,Jane Smith,Corporate,2,350.0,175.00
1,C003,Acme Corp,SMB,1,80.0,80.00
2,C010,Small Biz,SMB,1,45.0,45.00
3,C001,John Doe,Consumer,2,195.5,97.75
4,C005,Global LLC,Enterprise,2,800.0,400.00


In [84]:
query = """
with
active_orders as (
    select
        customer_id,
        count(*) as order_count
        sum(amount) as total_amount
        avg(amount) as avg_amount
    from 'orders.csv'
    where order_date >= date_add(current_date(), interval '-12 month')
    group by 1
),

active_customers as (
  select
      customer_id,
      customer_name,
      segment
    from 'new_customers.csv'
    where is_active = true

);

final_result as (
  select
  ac.*,
    COALESCE(ao.order_count, 0) as order_count,
    COALESCE(ao.total_amount, 0) as total_amount,
    COALESCE(ao.avg_amount, 0) as avg_amount,
  from active_customers ac
  left join active_orders ao
   on ac.customer_id = ao.customer_id
)

select * from final_result
"""

In [94]:
result2 = con.execute(query).fetchdf()

In [95]:
result2.head()

,order_id,customer_id,customer_name,order_date,amount,order_sequence,monthly_sales_rank,running_total,moving_avg_4_orders,previous_order,next_order,first_order_amount
0,O1001,C001,John Doe,2024-01-15,120.5,1,1,120.5,120.500000,0.0,75.0,120.5
1,O1002,C001,John Doe,2024-02-20,75.0,2,1,195.5,97.750000,120.5,50.0,120.5
2,O1003,C001,John Doe,2024-03-05,50.0,3,1,245.5,81.833333,75.0,0.0,120.5
3,O1004,C002,Jane Smith,2024-04-25,200.0,1,1,200.0,200.000000,0.0,150.0,200.0
4,O1005,C002,Jane Smith,2024-05-01,150.0,2,1,350.0,175.000000,200.0,0.0,200.0


In [89]:
query = """
WITH enriched_orders AS (
    SELECT
        o.*,
        c.customer_name,
        c.segment
    FROM orders.csv o
    LEFT JOIN new_customers.csv c
        ON o.customer_id = c.customer_id
)
SELECT
    order_id,
    customer_id,
    customer_name,
    order_date,
    amount,

    -- 1. ลำดับของ order ของลูกค้าแต่ละคน
    ROW_NUMBER() OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as order_sequence,

    -- 2. อันดับยอดขายในเดือนนั้น
    RANK() OVER (
        PARTITION BY strftime('%Y-%m', order_date)
        ORDER BY amount DESC
    ) as monthly_sales_rank,

    -- 3. ยอดสะสมของลูกค้า
    SUM(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as running_total,

    -- 4. ค่าเฉลี่ยเคลื่อนที่ 4 orders
    AVG(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) as moving_avg_4_orders,

    -- 5. ยอด order ก่อนหน้า
    LAG(amount, 1, 0) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as previous_order,

    -- 6. ยอด order ถัดไป
    LEAD(amount, 1, 0) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as next_order,

    -- 7. ยอด order แรกของลูกค้า
    FIRST_VALUE(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as first_order_amount

FROM enriched_orders
ORDER BY customer_id, order_date;
"""

window_analysis = con.execute(query).fetchdf()

In [90]:
window_analysis.head()

,order_id,customer_id,customer_name,order_date,amount,order_sequence,monthly_sales_rank,running_total,moving_avg_4_orders,previous_order,next_order,first_order_amount
0,O1001,C001,John Doe,2024-01-15,120.5,1,1,120.5,120.500000,0.0,75.0,120.5
1,O1002,C001,John Doe,2024-02-20,75.0,2,1,195.5,97.750000,120.5,50.0,120.5
2,O1003,C001,John Doe,2024-03-05,50.0,3,1,245.5,81.833333,75.0,0.0,120.5
3,O1004,C002,Jane Smith,2024-04-25,200.0,1,1,200.0,200.000000,0.0,150.0,200.0
4,O1005,C002,Jane Smith,2024-05-01,150.0,2,1,350.0,175.000000,200.0,0.0,200.0
